In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/推荐系统/推荐系统算法复现/ml-100k")
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [ ]:
# 导入user_df和item_df
user_df = pd.read_csv("./user_df.csv")
user_df = user_df.iloc[:,1:]
user_df.index += 1
item_df = pd.read_csv("./item_df.csv")
item_df = item_df.iloc[:,1:]
item_df.index += 1

In [ ]:
user_df.head()

,age,gender,occupation
1,2,9,29
2,5,8,23
3,2,9,30
4,2,9,29
5,3,8,23


In [ ]:
item_df.head()

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,30,31,32,34,35,36,36,37,38,39,40,41,42,43,44,45,46,47,48
2,30,32,33,33,34,35,36,37,38,39,40,41,42,43,44,45,47,47,48
3,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,47,48
4,30,32,32,33,34,36,36,37,39,39,40,41,42,43,44,45,46,47,48
5,30,31,32,33,34,35,37,37,39,39,40,41,42,43,44,45,47,47,48


In [ ]:
# 导入user-item交互矩阵
data = pd.read_csv("./user_item2.csv")
data.drop(data.columns[0], axis=1, inplace=True)
data.head()

,user_id,item_id,rating
0,1,55,1
1,1,203,1
2,1,183,1
3,1,150,1
4,1,68,1


In [ ]:
from PIL import Image
os.chdir("/content/drive/MyDrive/Colab Notebooks/推荐系统/推荐系统算法复现/图片")
image_path = "./deep crossing.png"
image = Image.open(image_path)
new_size = (700, 500)
resized_image = image.resize(new_size)
resized_image

In [9]:
import torch.nn.functional as F
class ResidualUnit(nn.Module):
    def __init__(self, input_size, output_size):
        super(ResidualUnit, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.fc2 = nn.Linear(output_size, output_size)
        self.downsample = nn.Linear(input_size, output_size) if input_size != output_size else None

    def forward(self, x):
        identity = x

        out = F.relu(self.fc1(x))
        out = self.fc2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.relu(out)

        return out

In [22]:
class deep_crossing(nn.Module):
    def __init__(self, num_features, factor_dim, res_layers_parameter_list, hidden_dims, user_df, item_df):
        super(deep_crossing, self).__init__()

        # 记录好用户和物品的特征索引
        self.user_df = user_df
        self.item_df = item_df

        # 嵌入层
        self.embeddings = nn.Embedding(num_features, factor_dim)

        # 残差网络层
        self.res_layers = nn.Sequential()
        for i in range(len(res_layers_parameter_list)):
            input_size = factor_dim * (user_df.shape[1]+item_df.shape[1]) if i == 0 else res_layers_parameter_list[i - 1]
            output_size = res_layers_parameter_list[i]
            res_unit = ResidualUnit(input_size, output_size)
            self.res_layers.add_module(f"residual_unit_{i}", res_unit)

        # 预测层
        self.predict_layer = nn.Linear(res_layers_parameter_list[-1], 1)

    #把用户和物品的特征合并起来
    def concat_user_item_vec( self, u, i ):
      # users的shape为(batch_size, users表格的特征个数)
      users = torch.LongTensor( self.user_df.loc[u].values )
      # items的shape为(batch_size, items表格的特征个数)
      items = torch.LongTensor( self.item_df.loc[i].values )
      # concat_vec的shape为(batch_size, users表格的特征个数+items表格的特征个数)
      concat_vec = torch.cat( [ users, items ], dim = 1 )
      return concat_vec

    def forward(self, u, i):
        # 得到用户与物品组合起来后的特征索引
        # concat_vec_index的shape为(batch_size, users表格的特征个数+items表格的特征个数)
        concat_vec_index = self.concat_user_item_vec( u, i )
        # 通过Embedding层，得到concat_vec_index中每个值对应的隐向量
        # embeddings的shape为(batch_size, users表格的特征个数+items表格的特征个数, hidden_dim )
        embeddings = self.embeddings( concat_vec_index )
        # embeddings的shape为(batch_size, (users表格的特征个数+items表格的特征个数)*hidden_dim )
        embeddings = embeddings.reshape(concat_vec_index.shape[0], -1)

        # 通过神经网络层
        # res_output的shape为(batch_size, final_res_layer_dim)
        res_output = self.res_layers(embeddings)

        # 预测
        output = self.predict_layer(res_output)
        return torch.squeeze(torch.sigmoid(output))

In [23]:
import numpy as np

# 设置随机数生成器的种子
np.random.seed(0)
# 创建一个1到10000的数组
arr = np.arange(data.shape[0])
# 原地随机排列数组
np.random.shuffle(arr)
# 选择百分之多少的数据作为训练集
train_test_ratio = 0.9
# 打乱后的数组的前百分之train_test_ratio(i.e.90%)作为训练集对应的index
train_index = arr[:int(len(arr)*train_test_ratio)]
# 打乱后的数组的后百分之(1-train_test_ratio)(i.e.10%)作为测试集对应的index
test_index = arr[int(len(arr)*train_test_ratio):]
# 获取训练集和测试集
train_set = data.iloc[train_index,:]
test_set = data.iloc[test_index, :]

# 将训练集和测试集的每一行都变成一个元组，方便后续处理
train_set = [tuple(row) for row in train_set.itertuples(index=False, name=None)]
test_set = [tuple(row) for row in test_set.itertuples(index=False, name=None)]

In [24]:
# 创建模型实例
num_features = item_df.values.max()+1
factor_dim = 64      # 嵌入的维度
num_layers = 1     # 神经网络层数
hidden_dims = [64]  # 每层的维度
res_layers_parameter_list = [32,16]

model = deep_crossing(num_features, factor_dim, res_layers_parameter_list, hidden_dims, user_df, item_df)

In [25]:
def evaluation(y_pred, y_true):
  p = precision_score(y_true, y_pred)
  r = recall_score(y_true, y_pred)
  acc = accuracy_score(y_true,y_pred)
  return p,r,acc

In [26]:
# 实例化损失函数和优化器
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score,recall_score,accuracy_score
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
  for user, item, rating in DataLoader(train_set, batch_size=512, shuffle=True):
    # 清空梯度
    optimizer.zero_grad()
    # 前向传播
    predictions = model(user, item)
    # 计算损失
    loss = loss_fn(predictions, rating.float())
    # 反向传播
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch}, Loss: {loss.item()}")

  # 训练集
  y_pred = np.array([1 if i >= 0.5 else 0 for i in predictions])
  y_true = rating.detach().numpy()
  precision, recall, acc = evaluation(y_pred, y_true)
  print('train: Precision {:.4f} | Recall {:.4f} | accuracy {:.4f}'.format(precision, recall, acc))

  #测试集
  user_test = torch.tensor(test_set)[:,0].detach()
  item_test = torch.tensor(test_set)[:,1].detach()
  predictions = model(user_test, item_test)
  y_pred = np.array([1 if i >= 0.5 else 0 for i in predictions])
  y_true = torch.tensor(test_set)[:,2].detach().float()
  precision, recall, acc = evaluation(y_pred, y_true)
  print('test: Precision {:.4f} | Recall {:.4f} | accuracy {:.4f}'.format(precision, recall, acc))
  print('----------------------------------------------------------------------------------------')

Epoch 0, Loss: 0.6801992058753967
train: Precision 0.5630 | Recall 0.7273 | accuracy 0.5625
test: Precision 0.6020 | Recall 0.7517 | accuracy 0.5830
----------------------------------------------------------------------------------------
Epoch 1, Loss: 0.6644997596740723
train: Precision 0.5984 | Recall 0.6995 | accuracy 0.5900
test: Precision 0.6071 | Recall 0.7541 | accuracy 0.5893
----------------------------------------------------------------------------------------
Epoch 2, Loss: 0.6733425855636597
train: Precision 0.5876 | Recall 0.5588 | accuracy 0.5750
test: Precision 0.6389 | Recall 0.5426 | accuracy 0.5725
----------------------------------------------------------------------------------------
Epoch 3, Loss: 0.6676933765411377
train: Precision 0.6304 | Recall 0.7468 | accuracy 0.5975
test: Precision 0.6048 | Recall 0.7557 | accuracy 0.5870
----------------------------------------------------------------------------------------
Epoch 4, Loss: 0.6567846536636353
train: Precisi